multiple enrolment
------------------------

현재 voxceleb은 1:1 trial만 제공하는데 실제 enrollment를 할 때 성능이 어떻게 변하는지?

enrollment 쪽에 아이디어가 있을 수 있나?

## Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
sys.path.append('/host/projects/sv_experiments/sv_system/')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [4]:
def key2df(keys, delimeter="-"):
    key_df = pd.DataFrame(keys, columns=['key'])
    key_df['spk'] = key_df.key.apply(lambda x: x.split(delimeter)[0])
    key_df['label'] = key_df.groupby('spk').ngroup()
    key_df['origin'] = key_df.spk.apply(lambda x: 'voxc2' if x.startswith('id') else 'voxc1')
    
    return key_df

In [6]:
import torch
from torch.nn.functional import cosine_similarity

def euc_dist(a, b):
    return ((a * a) + (b * b) - b * a).sum(1)

def cos_dist(a, b):
    a = a / np.linalg.norm(a, axis=1, keepdims=True)
    b = b / np.linalg.norm(b, axis=1, keepdims=True)
    return -(a*b).sum(1)

def cos_sim_batch(a, b, axis=2):
    a = torch.from_numpy(a)
    b = torch.from_numpy(b)
    
    return cosine_similarity(a, b, dim=axis).numpy()

### Baseline - average enrolled embeddings

In [5]:
si_keys = pickle.load(open("/host/projects/sv_experiments/sv_system/embeddings/voxc1/voxc1_mfcc30_best/si_keys.pkl", "rb"))
si_embeds = np.load("/host/projects/sv_experiments/sv_system/embeddings/voxc1/voxc1_mfcc30_best/si_embeds.npy")

sv_keys = pickle.load(open("/host/projects/sv_experiments/sv_system/embeddings/voxc1/voxc1_mfcc30_best/sv_keys.pkl", "rb"))
sv_embeds = np.load("/host/projects/sv_experiments/sv_system/embeddings/voxc1//voxc1_mfcc30_best/sv_embeds.npy")
 
trial = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_test_trial.pkl")

In [29]:
si_key_df = key2df(si_keys)
sv_key_df = key2df(sv_keys)

si_spks = si_key_df.spk.unique().tolist()
sv_spks = sv_key_df.spk.unique().tolist()

In [33]:
sv_key_df['session'] = sv_key_df.key.apply(lambda x: x[8:-6])

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
clf = LDA()
si_mean = si_embeds.mean(0)
clf.fit(si_embeds-si_mean, si_key_df.label)
overall_LDA = clf

In [70]:
lda_embeds = overall_LDA.transform(sv_embeds-si_mean)

In [95]:
enroll_session = dict()
for spk in sv_spks:
    enroll_session[spk] = np.random.choice(sv_key_df[sv_key_df.spk == spk].session, size=4, replace=False)

In [96]:
enrolled_aveg_embeds = []
enrolled_uttrs = []
for spk in sv_spks:
    sess = enroll_session[spk]
    enrolled_aveg_embeds += [lda_embeds[sv_key_df[sv_key_df.session.isin(sess)].index].mean(0)]
    enrolled_uttrs += sv_key_df[sv_key_df.session.isin(sess)].key.tolist()
enrolled_aveg_embeds = np.array(enrolled_aveg_embeds)

In [97]:
filtered_trial = trial[~trial.test_idx.isin(enrolled_uttrs)]

In [104]:
# now enrolment_id is not for uttrs but for spk
enrolment_id = filtered_trial.enroll_spk.apply(lambda x: sv_spks.index(x))
filtered_trial['enrolment_id'] = enrolment_id

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
from eval.score_utils import compute_eer
scores = cos_sim_batch(enrolled_aveg_embeds[filtered_trial.enrolment_id], 
                       lda_embeds[filtered_trial.test_id], axis=1)
compute_eer(scores[filtered_trial.label == 1], scores[filtered_trial.label == 0])

eer:3.141% at threshold 0.1469


(0.031409168081494056, 0.14690098121470396)

In [89]:
from eval.score_utils import compute_eer
scores = cos_sim_batch(lda_embeds[trial.enrolment_id], lda_embeds[trial.test_id], axis=1)
compute_eer(scores[trial.label == 1], scores[trial.label == 0])

eer:9.756% at threshold 0.0704


(0.09756097560975607, 0.07037735300434955)